## Etude 1 - The Game of Life (Cellular automata and computer graphics)

A LIFE community is a made up of cells that inhabit a square array. Every generation new cells may be born and old cells may die. A cell dies if it is lonely, that is it only has one neighbor (in the eight squares immediately surrounding it) or if it gets over crowded, if there are more than three neighbors. A new cell may be born only if there is an empty square with exactly three neighboring cells.

Challenge: Write a program that simulates a LIFE community. The input should be the initial positions of the communitiy's cells and the putput and aerial view of the community at each generation. 

Notes: Some communities grow enormously from small beginnings.  Others transport themselves slowly across the plane. Your program should be able to handle large communities without a terrific cost in space or time.


###  Initial thoughts

In this first go I've not read any related literature on cellular automata or done any performance tuning.

Keeping everything in an array will likely include a lot of irrelivant squares. Given a set of active points (xy coordinates) you have the initial active points, all of their neighboring points and if any of those neighboring points are empty, their neighboring points. You need that last category to evaluate the potential new cell locations. 
Therefore the max number of locations you need to keep track of is 25n for n active locations. In all likelyhood however many of these locations will overlap. 

The mechanics of the simulation are modeled in the LIFEstate class and the animation details are handled in the Animate class which is then passed to the Matplotlib function FuncAnimation.

In [1]:
import numpy as np
import pandas as pd
import itertools
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation

In [2]:
class LIFEstate(object):
    def __init__(self, starting_locations):
        self.locations = starting_locations
        
    def get_neighbors(self, location):
        for n , m in itertools.product([-1, 0, 1], [-1, 0, 1]):
            if  (n == 0 and m == 0):
                continue
            yield location[0] + n, location[1] + m
            
    def get_all_locations(self, active_locations):
        nearby_locations = dict((k,1) for k in active_locations)
        for location in active_locations:  
            for neighbor in self.get_neighbors(location):
                if (neighbor in active_locations):
                    continue
                nearby_locations[neighbor] = 0
            
        all_locations = nearby_locations.copy()
        for location in nearby_locations:
            if(nearby_locations[location] == 1):
                continue
            for neighbor in self.get_neighbors(location):
                if(neighbor in nearby_locations):
                    continue
                all_locations[neighbor] = -1
        return all_locations    

    def evaluate_next_generation(self, all_locations):
        new_active_locations = {}
        for location in all_locations:
            if(all_locations[location] == -1):
                continue
            if(all_locations[location] == 0):
                count = 0
                for neighbor in self.get_neighbors(location):
                    if(all_locations[neighbor] == 1):
                        count += 1
                if(count == 3):
                    new_active_locations[location] = 1
            if(all_locations[location] == 1):
                count = 0
                for neighbor in self.get_neighbors(location):
                    if(all_locations[neighbor] == 1):
                        count += 1
                if(count == 2 or count == 3):
                    new_active_locations[location] = 1
        return list(new_active_locations.keys())
    
    def get_next_generation(self):
        all_locations = self.get_all_locations(self.locations)
        new_active_locations = self.evaluate_next_generation(all_locations)
        self.locations = new_active_locations
        return self.locations

In [57]:
class Animate(object):
    def __init__(self, ax, LIFEstate):
        self.lifestate = LIFEstate
        self.x, self.y = zip(*self.lifestate.locations)
        self.frame, = ax.plot([],[], 'o', markersize = 17) 
        self.ax = ax
        self.padding = 3
    
    def adjust_view(self):
        xmin = np.min(self.x) - self.padding
        xmax = np.max(self.x) + self.padding
        ymin = np.min(self.y) - self.padding
        ymax = np.max(self.y) + self.padding
        
        self.ax.set_xlim(xmin, xmax)
        self.ax.set_ylim(ymin, ymax)
        
    
    def __call__(self, i):
        try:
            self.x, self.y = zip(*self.lifestate.get_next_generation())
        except ValueError:
            self.frame.set_data([],[]) 
            return self.frame,

        self.adjust_view()
        self.frame.set_data(self.x,self.y)
        
        return self.frame,


In [59]:
fig, ax = plt.subplots()
lifestate = LIFEstate([(0,0), (1,0), (1,1), (1, -1), (2,0), (2,1), (2,2)])
a = Animate(ax, lifestate)
anim = FuncAnimation(fig, a, frames=np.arange(100), interval=100, blit=True)
plt.show()

In [ ]:
#Some interesting starts
# [(0,0), (0,1), (1,1), (0,2), (2,2), (3,3), (4,4), (5,5)]
# [(0,0), (1,0), (1,1), (1, -1), (2,0), (2,1), (2,2)]
#[(0,0), (0,1), (0,2),(0,3), (0,4), (0,5),(1,0), (1,1), (1,2),(1,3), (1,4), (1,5), (2,0), (2,1), (2,2),(2,3), (2,4), (2,5)]

### Thoughts for refinement / other features:

I'm making heavy use of the generator get_neighbors and then looking up locations in a dictionary. I wonder if there is some sort of graph structure that might better store the relationships rather than generating the key and looking it up every time. 

Would like to add some better analysis for finding interesting evolutions - things like finding steady states or patterns that behave in predictable ways. Perhaps other ways of coloring the animation that give more intuition on the process.

Also the Animate class is muddy, has some duplicate code and ugly looking type translations

#### References listed in book:

Burks, Aurthur W. (Ed.) Essays on Cellular Automata. University of Illinois Press, Urbana, IL, 1970

Codd, E. F. Cellular Automata, Academic Press, New York, NY, 1968

Gardner, Martin "Mathematical Games" Scientific American, 223, 10, pp. 120-123, October 1970, and 224, 2, pp. 112-117, February 1971

Wainwright, Rovert T. (Ed.) Lifeline, 1280 Edcris Road,  Yorktown Heights, NY 10598.